In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from mlserve_sdk.client import MLServeClient
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# -----------------------------
# Fraud Data Generator
# -----------------------------
def generate_fraud_data(n_samples=1000, missing_frac=0.05, random_state=42):
    """
    Generate synthetic fraud dataset for ML benchmarking.

    Parameters
    ----------
    n_samples : int
        Number of rows to generate.
    missing_frac : float
        Fraction of missing values to inject per column (0–1).
    random_state : int
        Seed for reproducibility.

    Returns
    -------
    X : pd.DataFrame
        Feature matrix with categorical, numerical & text features.
    y : pd.Series
        Binary fraud target (0 = legit, 1 = fraud).
    """
    np.random.seed(random_state)

    data = {
        "transaction_id": np.arange(1, n_samples + 1),
        "amount": np.round(np.random.exponential(scale=100, size=n_samples), 2),
        "transaction_type": np.random.choice(
            ["online_purchase", "in_store", "transfer", "atm_withdrawal"],
            size=n_samples,
            p=[0.4, 0.3, 0.2, 0.1],
        ),
        "device_type": np.random.choice(
            ["mobile", "desktop", "tablet"],
            size=n_samples,
            p=[0.6, 0.3, 0.1],
        ),
        "customer_age": np.random.randint(18, 80, n_samples),
        "account_tenure_days": np.random.randint(30, 3650, n_samples),
        "country": np.random.choice(
            ["US", "UK", "DE", "NG", "IN", "CN", "BR"],
            size=n_samples,
        ),
        "num_prev_transactions": np.random.poisson(lam=30, size=n_samples),
    }

    X = pd.DataFrame(data)

    # Inject missing values
    if missing_frac > 0:
        for col in X.columns.drop("transaction_id"):
            X.loc[
                X.sample(frac=missing_frac, random_state=random_state).index, col
            ] = np.nan

    # Fraud probability (rules + noise)
    prob_fraud = (
        0.3 * (X["transaction_type"] == "online_purchase").astype(float)
        + 0.25 * (X["device_type"] == "mobile").astype(float)
        + 0.2 * (X["country"].isin(["NG", "CN", "BR"])).astype(float)
        + 0.002 * X["amount"].fillna(100)
        + 0.01 * (X["num_prev_transactions"].fillna(0) < 5).astype(float)
        + np.random.normal(0, 0.1, n_samples)
    )
    prob_fraud = 1 / (1 + np.exp(-prob_fraud))  # sigmoid

    y = pd.Series(np.random.binomial(1, prob_fraud), name="is_fraud")

    return X, y

In [4]:
X, y = generate_fraud_data()
# Drop ID column if exists
if "transaction_id" in X.columns:
    X = X.drop(columns=["transaction_id"])

categorical_features = ["transaction_type", "device_type", "country"]
numeric_features = ["amount", "customer_age", "account_tenure_days", "num_prev_transactions"]

# -----------------------------
# ColumnTransformer
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features),
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), numeric_features)
    ]
)

# -----------------------------
# Full pipeline
# -----------------------------
model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(eval_metric="logloss"))
])
# Ensure X is DataFrame
X = X.copy()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model.fit(X_train, y_train)

# Evaluate
print("Train score:", model.score(X_train, y_train))
print("Test score:", model.score(X_test, y_test))

Train score: 1.0
Test score: 0.565


In [5]:
# -----------------------------
# 2. Connect to MLServe
# -----------------------------
USERNAME = os.getenv("USERNAME")
TOKEN = os.getenv("TOKEN")

client = MLServeClient()
client.login(USERNAME, TOKEN)

In [6]:
try:
    lv=client.get_latest_version("fraud")
    next_version=lv["next_version"]
except:
    next_version="v1"

print(next_version)

v1


In [7]:
# -----------------------------
# 3. Deploy model
# -----------------------------
feats=list(X)

client.deploy(
    model=model,
    name="fraud",
    version=next_version,
    features=feats,
    background_df=X.sample(300),
    metrics={'accuracy':model.score(X_test, y_test)},
    task_type='classification'
)

{'predict_url': 'https://mlserve.com/api/v1/predict/fraud/v1'}

In [8]:
%%time

TEST_DATA = {
    "features": X.columns.tolist(),
    "inputs": X.values.tolist()
}

preds = client.predict("fraud", next_version, TEST_DATA, explain=True)
#preds['explanations']

CPU times: user 25 ms, sys: 2.8 ms, total: 27.8 ms
Wall time: 697 ms


In [9]:
%%time

TEST_DATA = {
    "features": X.columns.tolist(),
    "inputs": X.values.tolist()
}
preds = client.predict_weighted("fraud", TEST_DATA, entity_ids=["user-133"]*len(X))

CPU times: user 25.6 ms, sys: 2.58 ms, total: 28.2 ms
Wall time: 904 ms


In [10]:
# -----------------------------
# 7. Fetch Metrics
# -----------------------------
metrics = client.get_metrics("fraud", "v1", as_dataframe=True)
metrics

,requests,predictions,throughput_rps,prediction_rps,avg_latency_ms,p50_latency_ms,p95_latency_ms,p99_latency_ms,avg_latency_per_element_ms,p50_latency_per_element_ms,p95_latency_per_element_ms,p99_latency_per_element_ms,error_rate
timestamp,,,,,,,,,,,,,
2025-09-11 20:00:00,2,2000,0.000556,0.555556,127.894094,127.894094,135.712614,136.407594,0.127894,0.127894,0.135713,0.136408,0.0


In [11]:
# -----------------------------
# 7. Fetch Data Quality Metrics
# -----------------------------
d = client.get_data_quality("fraud", "v1", as_dataframe=True)

In [12]:
d['missingness']

,feature,missing_fraction,invalid_fraction
0,amount,0.05,0.0
1,transaction_type,0.05,0.0
2,device_type,0.05,0.0
3,customer_age,0.05,0.0
4,account_tenure_days,0.05,0.0
5,country,0.05,0.0
6,num_prev_transactions,0.05,0.0


In [13]:
d['drift']

,feature,psi,ks,wasserstein,pct_mean_diff,status
0,amount,0.005318,0.020134,3.024839,-0.814108,alert
1,customer_age,0.019286,0.035215,0.823367,1.636819,alert
2,account_tenure_days,0.030991,0.068652,111.136357,-5.749081,alert
3,num_prev_transactions,0.021648,0.049332,0.550795,1.810694,alert


In [14]:
d['outliers']

,feature,iqr_fraction,zscore_fraction,status
0,amount,0.038,0.017895,ok
1,customer_age,0.000,0.000000,ok
2,account_tenure_days,0.000,0.000000,ok
3,num_prev_transactions,0.003,0.003158,ok


In [15]:
client.list_models()

[{'model': 'synth',
  'created_at': '2025-09-11T20:45:09.094906',
  'versions': [{'version': 'v2',
    'weight': 1.0,
    'internal_url': 'http://synth-v2:8080/predict',
    'url': 'http://localhost:53927/predict',
    'active': True,
    'deployed_at': '2025-09-11T20:45:26.963827',
    'features': ['age',
     'tenure_months',
     'monthly_charges',
     'total_charges',
     'contract_type',
     'payment_method',
     'internet_service',
     'gender',
     'has_phone_service',
     'num_dependents'],
    'model_metadata': {'model_class': 'XGBClassifier',
     'module': 'xgboost.sklearn',
     'top_level': {'class': 'XGBClassifier',
      'module': 'xgboost.sklearn',
      'structure': {'name': None,
       'class': 'XGBClassifier',
       'module': 'xgboost.sklearn',
       'type': 'xgboost',
       'params': {'objective': 'binary:logistic',
        'base_score': None,
        'booster': None,
        'callbacks': None,
        'colsample_bylevel': None,
        'colsample_bynode'

In [16]:
# now let's give feedback for 10 more predictions
test_ids=preds["prediction_ids"][:20]

feedback=[]
for tid in test_ids:
    val=np.random.randint(0, 2)
    r=np.random.normal(10, 7)
    feedback.append({"prediction_id":tid, "true_value":val, "reward":r})

client.send_feedback(feedback)

{'status': 'ok', 'updated': 20, 'not_found': []}

In [17]:
client.get_online_metrics("fraud", "v1", window_hours=24, as_dataframe=True)

,model,version,window_hours,n,n_supervised,accuracy,f1,brier,mean_reward,n_rewards
0,fraud,v1,24,2000,20,0.4,0.454545,0.6,8.453309,20


In [18]:
# I don't new v1 anymore. Let's stop it
client.stop_model("fraud", "v1", remove=True)

{'status': 'ok',
 'message': 'Successfully stopped fraud:v1 and removed its container and image'}